In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from riotwatcher import LolWatcher, ApiError


In [108]:
lol_watcher = LolWatcher('RGAPI-a2e5141b-98d9-427a-905c-594767fb16cf')
my_region = 'euw1'


In [95]:
def liste_player_all_ranks():
    list_finale = []
    list_finale = pd.DataFrame(list_finale, columns=['puuid'])
    random_page = np.random.randint(1, 10)
    for tier in ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']:
        for division in ['I', 'II', 'III', 'IV']:
            try:
                listloc = lol_watcher.league.entries(my_region,
                                                   queue='RANKED_SOLO_5x5',
                                                   tier=tier,
                                                   division=division,
                                                   page=random_page)
                listloc = pd.DataFrame(listloc)
                listloc = pd.DataFrame(listloc['puuid'].drop_duplicates().reset_index(drop=True))
                listloc.columns = ['puuid']
                list_finale = pd.concat([list_finale, listloc], ignore_index=True)
                print(f"Loaded {len(listloc)} players from {tier} {division}")
            except ApiError as err:
                print(f"Error fetching data for {tier} {division}: {err}")
    
    try:
        listlocchall = lol_watcher.league.challenger_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocchall = pd.DataFrame(listlocchall)
        listchallpuuid = listlocchall['entries'].apply(lambda x: x['puuid'])
        listlocchall = pd.DataFrame(listchallpuuid).drop_duplicates().reset_index(drop=True)
        listlocchall.columns = ['puuid']
        listlocchall = pd.DataFrame(listlocchall['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocchall.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocchall], ignore_index=True)
        print(f"Loaded {len(listlocchall)} players from challenger")
    except ApiError as err:
        print(f"Error fetching data for challenger: {err}")
    
    try:
        listlocmaster = lol_watcher.league.masters_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocmaster = pd.DataFrame(listlocmaster)
        listmasterpuuid = listlocmaster['entries'].apply(lambda x: x['puuid'])
        listlocmaster = pd.DataFrame(listmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocmaster.columns = ['puuid']
        listlocmaster = pd.DataFrame(listlocmaster['puuid'].sample(410).drop_duplicates().reset_index(drop=True))
        listlocmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocmaster], ignore_index=True)
        print(f"Loaded {len(listlocmaster)} players from master")
    except ApiError as err:
        print(f"Error fetching data for master: {err}")

    try:
        listlocgrandmaster = lol_watcher.league.grandmaster_by_queue(my_region, 'RANKED_SOLO_5x5')
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster)
        listgrandmasterpuuid = listlocgrandmaster['entries'].apply(lambda x: x['puuid'])
        listlocgrandmaster = pd.DataFrame(listgrandmasterpuuid).drop_duplicates().reset_index(drop=True)
        listlocgrandmaster.columns = ['puuid']
        listlocgrandmaster = pd.DataFrame(listlocgrandmaster['puuid'].sample(205).drop_duplicates().reset_index(drop=True))
        listlocgrandmaster.columns = ['puuid']
        list_finale = pd.concat([list_finale, listlocgrandmaster], ignore_index=True)
        print(f"Loaded {len(listlocgrandmaster)} players from grand master")
    except ApiError as err:
        print(f"Error fetching data for grand master: {err}")
    
    list_finale = list_finale.drop_duplicates().reset_index(drop=True)
    return list_finale


In [135]:
df_test = liste_player_all_ranks()

Loaded 205 players from IRON I
Loaded 205 players from IRON II
Loaded 205 players from IRON III
Loaded 205 players from IRON IV
Loaded 205 players from BRONZE I
Loaded 205 players from BRONZE II
Loaded 205 players from BRONZE III
Loaded 205 players from BRONZE IV
Loaded 205 players from SILVER I
Loaded 205 players from SILVER II
Loaded 205 players from SILVER III
Loaded 205 players from SILVER IV
Loaded 205 players from GOLD I
Loaded 205 players from GOLD II
Loaded 205 players from GOLD III
Loaded 205 players from GOLD IV
Loaded 205 players from PLATINUM I
Loaded 205 players from PLATINUM II
Loaded 205 players from PLATINUM III
Loaded 205 players from PLATINUM IV
Loaded 205 players from DIAMOND I
Loaded 205 players from DIAMOND II
Loaded 205 players from DIAMOND III
Loaded 205 players from DIAMOND IV
Loaded 205 players from challenger
Loaded 410 players from master
Loaded 205 players from grand master


In [145]:
df_test.head()

,puuid
0,UEgCPIQwqomxJO8fr__NA6SPtASX-gyzqynrd5T7FE8z0a...
1,sSr0hDquRqOlGeO5oqR24CXZ9LeSLb_B8gXhdMQlAxGCG2...
2,O435vOI81ZxLb2cqxi_q-0N2y_b_ge4KOEkxhfdpj47VsC...
3,eKAxixfYcOfBmGc9o0wR-IjQ8FlnfwSfB15idqKX_xGyDb...
4,DxmmF5Ye378Mp_GkQK7CWg0pbI6yrKW9qHIRNqAn0eHkyz...


In [ ]:
def last_ranked_games_by_puuid(puuidloc, nbgames):
    try:
        matchlist = lol_watcher.match.matchlist_by_puuid(region = my_region,
                                                         puuid = puuidloc,
                                                         queue=420, # RANKED_SOLO_5x5
                                                         type='ranked', #optional
                                                         count=nbgames)
        return matchlist  
    except ApiError as err:
        print(f"Error fetching match list for {puuidloc}: {err}")
    return None

In [158]:
def create_last_games_df(puuid_df, nbgames):
    last_games = []
    for puuid in puuid_df['puuid']:
        last_games_id = last_ranked_games_by_puuid(puuid, nbgames)
        if last_games_id:
            last_games.append(last_games_id)
        else:
            last_games.append(None)
    last_games_df = pd.DataFrame(last_games)
    last_games_df = last_games_df.dropna().reset_index(drop=True)
    last_games_df = last_games_df.rename(columns={0: 'game_id'})
    last_games_df = last_games_df.set_index('game_id')
    return last_games_df

In [ ]:
def did_player_won_game(region, puuid, game_id):
    try:
        game = lol_watcher.match.by_id(region, game_id)
        for participant in game['info']['participants']:
            if participant['puuid'] == puuid:
                return participant['win']
    except ApiError as err:
        print(f"Error fetching game data: {err}")
    return False

In [171]:

def get_match_players_roles(region, game_id):
    match_info = lol_watcher.match.by_id(region, game_id)
    participants = match_info['info']['participants']

    # Map teamId → side
    side_map = {100: "BlueSide", 200: "RedSide"}

    # Initialize dict for this game
    game_data = {"gameId": game_id}

    for p in participants:
        side = side_map[p["teamId"]]

        # Lane/role from Riot
        lane = p["lane"]
        role = p["role"]

        # Normalize into standard LoL positions
        if lane == "TOP":
            position = "Top"
        elif lane == "JUNGLE":
            position = "Jungle"
        elif lane == "MIDDLE":
            position = "Mid"
        elif lane == "BOTTOM":
            position = "ADC" if role == "DUO_CARRY" else "Support"
        elif lane == "UTILITY":
            position = "Support"
        else:
            position = lane  # fallback (in case Riot returns weird stuff)

        # Build column names
        puuid_col = f"{side}{position}_puuid"

        # Fill the dict
        game_data[puuid_col] = p["puuid"]
    
    # Add victorious side
    teams_info = match_info['info']['teams']
    if teams_info[0]['win']:   # teamId 100 → BlueSide
        game_data["TeamVictorious"] = "BlueSide"
    else:
        game_data["TeamVictorious"] = "RedSide"

    return pd.DataFrame([game_data])


In [ ]:
def stats_of_player_before_game(region, puuid, game_id):
    
    